<a href="https://colab.research.google.com/github/VanBap/Adverse_Weather/blob/master/Test_model_using_ResNet50_tflite.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Ngô Hoàng Khánh Văn - 20021599

from tensorflow import keras

from tensorflow.keras.models import Model

from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image


import tensorflow as tf
import os
import matplotlib.pyplot as plt
import numpy as np
import glob
import pathlib
import pandas as pd

In [2]:
# Run this cell if u can't mount ur drive automatically
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
train_data_dir = '/content/drive/MyDrive/KHOA_LUAN/DATASET_splitted/train'
valid_data_dir = '/content/drive/MyDrive/KHOA_LUAN/DATASET_splitted/val'
test_data_dir = '/content/drive/MyDrive/KHOA_LUAN/DATASET_splitted/test'

img_height, img_width = (224, 224)
batch_size = 32

train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True
                                   )
# tạo dữ liệu tăng cường: xoay ảnh = 0.2; phóng to = 0.2; lật ảnh ngang; giữ 40% cho validation

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size = batch_size,
    class_mode='categorical')
    #subset='training') # set as training data

valid_generator = train_datagen.flow_from_directory(
    valid_data_dir,
    target_size=(img_height, img_width),
    batch_size = batch_size,
    class_mode='categorical')
    #subset='validation') # set as validation data

test_generator = train_datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_height, img_width),
    batch_size = 1,
    class_mode='categorical')
    #subset='validation') # set as testing data

train_generator.class_indices
valid_generator.class_indices
test_generator.class_indices

Found 1289 images belonging to 7 classes.
Found 428 images belonging to 7 classes.
Found 435 images belonging to 7 classes.


{'Cloudy': 0,
 'Fog': 1,
 'Rainy': 2,
 'Sandy': 3,
 'Shine': 4,
 'Snowy': 5,
 'Sunrise': 6}

**Test độ chính xác của model .tflite sau khi được lượng tử hóa float16**




In [4]:
import tensorflow as tf

# Load model
model_path = '/content/drive/MyDrive/KHOA_LUAN/saved_model/ResNet50_Weather_epoch20_optimizing_date24.tflite'
interpreter = tf.lite.Interpreter(model_path=model_path)
interpreter.allocate_tensors()

# Get input and output details
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
# Resize Tensor Shape
interpreter.resize_tensor_input(input_details[0]['index'], (1, 224, 224, 3))
#interpreter.resize_tensor_input(output_details[0]['index'], (1, 7))
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
# Perform evaluation
print("input_details là: ", input_details)
print(output_details)
correct_predictions = 0
total_samples = 0

for images, labels in test_generator:
    interpreter.set_tensor(input_details[0]['index'], images)
    interpreter.invoke()
    output_data = interpreter.get_tensor(output_details[0]['index'])
    predictions = tf.argmax(output_data, axis=1)
    correct_predictions += tf.reduce_sum(tf.cast(tf.equal(predictions, labels), dtype=tf.int32))
    total_samples += len(labels)

    # Thêm điều kiện để thoát khỏi vòng lặp khi đã thử nghiệm hết tất cả các mẫu
    if total_samples >= len(test_generator.filenames):
        break


input_details là:  [{'name': 'serving_default_resnet50_input:0', 'index': 0, 'shape': array([  1, 224, 224,   3], dtype=int32), 'shape_signature': array([-1, -1, -1,  3], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]
[{'name': 'StatefulPartitionedCall:0', 'index': 299, 'shape': array([1, 7], dtype=int32), 'shape_signature': array([-1,  7], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]


In [10]:
# Calculate accuracy
test_accuracy = correct_predictions / total_samples
print('Total samples:', total_samples)
print('Correct predictions:', correct_predictions)
print('Test accuracy:', test_accuracy.numpy())

Total samples: 435
Correct predictions: tf.Tensor(409, shape=(), dtype=int32)
Test accuracy: 0.9402298850574713
